In [ ]:
import os
import time
import pandas as pd
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Thư mục chứa các file CSV gốc
input_folder = "/Users/macbook/Downloads/BigData/DataStream"

# Thư mục chứa các file CSV đã xử lý
output_folder = "/Users/macbook/Downloads/BigData/DataStream_cleaned"
os.makedirs(output_folder, exist_ok=True)

# Các cột cần giữ lại
columns_to_keep = ['FTR', 'AwayTeam', 'AST', 'Date']

class CSVHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.endswith(".csv"):
            return
        self.process(event.src_path)

    def on_modified(self, event):
        if event.is_directory or not event.src_path.endswith(".csv"):
            return
        self.process(event.src_path)

    def process(self, filepath):
        filename = os.path.basename(filepath)
        output_path = os.path.join(output_folder, filename)
        try:
            df = pd.read_csv(filepath)

            # Kiểm tra nếu cột Date tồn tại và xử lý
            if 'Date' in df.columns:
                # Kiểm tra và xử lý trường hợp ngày có định dạng dd/MM/yy
                df['Date'] = df['Date'].apply(self.correct_year_format)

                # Chuyển cột Date sang kiểu datetime sau khi đã chỉnh sửa năm
                df['Date'] = pd.to_datetime(df['Date'], format="%d/%m/%Y", errors='coerce')

                # Nếu có giá trị không hợp lệ, giữ lại giá trị gốc (có thể thay bằng giá trị mặc định như "NaT")
                df['Date'] = df['Date'].fillna(df['Date'])

                # Chỉ giữ dữ liệu từ năm 2018 đến 2020
                df = df[df['Date'].dt.year.between(2019, 2020)]

                # Chuyển định dạng ngày về yyyy-MM-dd
                df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

            # Lọc các cột cần giữ lại
            df_filtered = df[columns_to_keep]
            df_filtered.to_csv(output_path, index=False)
            print(f"✅ Đã xử lý và lọc file: {filename}")
        except Exception as e:
            print(f"❌ Lỗi khi xử lý {filename}: {e}")

    def correct_year_format(self, date):
        """
        Hàm này sẽ kiểm tra và sửa định dạng năm nếu ngày có dạng 'dd/MM/yy' thành 'dd/MM/yyyy'.
        """
        try:
            # Kiểm tra nếu năm là 2 chữ số (dd/MM/yy)
            if len(date.split("/")[2]) == 2:
                return date[:6] + '20' + date[6:]  # Thêm "20" vào đầu năm
            return date
        except Exception as e:
            print(f"❌ Lỗi khi xử lý ngày: {date} - {e}")
            return date  # Nếu có lỗi, trả lại ngày gốc

if __name__ == "__main__":
    event_handler = CSVHandler()
    observer = Observer()
    observer.schedule(event_handler, path=input_folder, recursive=False)
    observer.start()
    print(f"👀 Đang theo dõi thư mục: {input_folder}")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        observer.stop()
    observer.join()


👀 Đang theo dõi thư mục: /Users/macbook/Downloads/BigData/DataStream
✅ Đã xử lý và lọc file: SP1_2017_2018.csv
✅ Đã xử lý và lọc file: SP1_2020_2021.csv
✅ Đã xử lý và lọc file: SP1_2019_2020.csv
✅ Đã xử lý và lọc file: SP1_2017_2018.csv
✅ Đã xử lý và lọc file: SP1_2020_2021.csv
✅ Đã xử lý và lọc file: SP1_2019_2020.csv
